In [ ]:
import yfinance as yf 

### All of the following necessary for Elastic Net Regression
import numpy as np 
import pandas as pd 
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

## Download

In [ ]:
tickers = ['AAPL', 'AMZN', 'MSFT', 'QQQ', '^GSPC']
df = yf.download(tickers, start = '2020-01-01', end='2025-04-01')['Close']
df.head()

## Feature Engineering

In [ ]:
#Yesterday's values
df['AAPL(t-1)'] = df['AAPL'].shift(1)
df['AMZN(t-1)'] = df['AMZN'].shift(1)
df['MSFT(t-1)'] = df['MSFT'].shift(1)
df['QQQ(t-1)'] = df['QQQ'].shift(1)
df['^GSPC(t-1)'] = df['^GSPC'].shift(1)

#Moving Average (MA): Indicator for short term trend 
df['AAPL_MA_5'] = df['AAPL'].rolling(window = 5).mean()
df['AMZN_MA_5'] = df['AMZN'].rolling(window = 5).mean()
df['MSFT_MA_5'] = df['MSFT'].rolling(window = 5).mean()
df['QQQ_MA_5'] = df['QQQ'].rolling(window = 5).mean()
df['^GSPC_MA_5'] = df['^GSPC'].rolling(window = 5).mean()

#Target variable 
df['Target'] = df['AAPL'].shift(-1)

df = df.dropna()

## Elastic Net Regression 

### 1) Define Features and Target Variables

In [ ]:
X = df[['AAPL(t-1)', 'AMZN(t-1)', 'MSFT(t-1)', 'QQQ(t-1)', '^GSPC(t-1)',
        'AAPL_MA_5', 'AMZN_MA_5', 'MSFT_MA_5', 'QQQ_MA_5', '^GSPC_MA_5']]

Y = df['Target']

### 2) Train Test Split

In [ ]:
# 95.6% training and 4.4% testing 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.044, shuffle=False)

### 3) Apply Elastic Net Regression

In [ ]:
elasticnet = ElasticNet(alpha = 1, l1_ratio = 0.5) #50% lasso 50% ridge
elasticnet.fit(X_train, Y_train)

### 4) Get intercept and Coeff for Elastic Net Regression 

In [ ]:
coefficients = elasticnet.coef_ 

intercept = elasticnet.intercept_

coeff_df = pd.DataFrame({'Feature':X.columns, 'Coefficients':coefficients})
#coeff_df.head()

### 5) Predict using Elastic Net Regression

In [ ]:
y_pred = elasticnet.predict(X_test)

### 6) Create a dataframe with Actual and Predicted Values 

In [ ]:
df_result = pd.DataFrame({'Actual': Y_test, 'Predicted': y_pred})

### 7) Plot Actual & Predicted Values 

In [ ]:
plt.figure(figsize = (14,6))
plt.plot(df_result.index, df_result['Actual'], label = 'Actual', color = 'green')
plt.plot(df_result.index, df_result['Predicted'], label = 'Predicted', color = 'red')
plt.title("Actual vs Prediction for AAPL Stock (2025) - Elastic Net Regression")
plt.xlabel("Date")
plt.ylabel("Stock Price")
plt.tight_layout()
plt.grid(True)
plt.show()

### 8) Evaluate the model - R square, mse, rmse

In [ ]:
r2 = r2_score(Y_test, y_pred)
print("R square", r2)

mse = mean_squared_error(Y_test, y_pred)
print("mse", mse)

rmse = np.sqrt(mse)
print("rmse", rmse)